# Decision Trees and Ensemble Learning: From Scratch to Modern Boosting

## 1. Работа с данными
------

### Подключение библиотек

In [18]:
!pip install category_encoders # В google Colab не была установлена данная библиотека
!pip install catboost # В google Colab не была установлена данная библиотека

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.8 MB/s eta 0:00:00


In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
from category_encoders import CountEncoder
import gc
import psutil
import scipy.special
import lightgbm as lgb
import catboost as cb
import xgboost as xgb

from datetime import datetime

### Загрузка данных

In [ ]:
# Нужно указать релевантные пути к файлам
train_path = "training.csv" 
test_path = "test.csv"

train_df = pd.read_csv(train_path)

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   RefId                              72983 non-null  int64  
 1   IsBadBuy                           72983 non-null  int64  
 2   PurchDate                          72983 non-null  object 
 3   Auction                            72983 non-null  object 
 4   VehYear                            72983 non-null  int64  
 5   VehicleAge                         72983 non-null  int64  
 6   Make                               72983 non-null  object 
 7   Model                              72983 non-null  object 
 8   Trim                               70623 non-null  object 
 9   SubModel                           72975 non-null  object 
 10  Color                              72975 non-null  object 
 11  Transmission                       72974 non-null  obj

### Разделение данных на train, validation и test

In [5]:
train_df['PurchDate'] = pd.to_datetime(train_df['PurchDate'])
train_df = train_df.sort_values('PurchDate').reset_index(drop=True)

In [6]:
n = len(train_df)
train_end = int(n * 0.33)
valid_end = int(n * 0.66)

train = train_df.iloc[:train_end]
valid = train_df.iloc[train_end:valid_end]
test = train_df.iloc[valid_end:]

y_train = train['IsBadBuy']
y_valid = valid['IsBadBuy']
y_test = test['IsBadBuy']

X_train = train.drop(['IsBadBuy', 'PurchDate'], axis = 1)
X_valid = valid.drop(['IsBadBuy', 'PurchDate'], axis = 1)
X_test = test.drop(['IsBadBuy', 'PurchDate'], axis = 1)

**OneHotEncoder**

Преобразует категориальные признаки в бинарные столбцы. Параметр handle_unknown='ignore' присваивает нули новым категориям в валидационной/тестовой выборках, предотвращая ошибки. Это увеличивает размерность данных (особенно если много категорий), но подходит для моделей, устойчивых к высокой размерности (например, деревья).

- **Плюсы:** Простота интерпретации, подходит для большинства моделей, обрабатывает новые категории через handle_unknown='ignore'.
- **Минусы:** Увеличивает размерность данных (например, столбец "Make" с 50 уникальными значениями создаёт 50 новых столбцов), что может замедлить обучение на больших датасетах.
- **Когда использовать:** Если датасет небольшой или модель (например, CatBoost) не имеет встроенной обработки категорий.

In [7]:
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)
    ], remainder='passthrough'
)

X_train_enc = preprocessor.fit_transform(X_train)
X_valid_enc = preprocessor.transform(X_valid)
X_test_enc = preprocessor.transform(X_test)

feature_names = preprocessor.get_feature_names_out()
X_train_enc = pd.DataFrame(X_train_enc, columns=feature_names)
X_valid_enc = pd.DataFrame(X_valid_enc, columns=feature_names)
X_test_enc = pd.DataFrame(X_test_enc, columns=feature_names)

print(f'Размер train: {X_train_enc.shape}, valid: {X_valid_enc.shape}, test: {X_test_enc.shape}')

Размер train: (24084, 1713), valid: (24084, 1713), test: (24815, 1713)


**CountEncoder**

Альтернатива, заменяющая категории их частотой в тренировочной выборке. Новые категории получают значение 1 (handle_unknown=1). Это сохраняет меньшую размерность, что полезно для больших датасетов.

- **Плюсы:** Сохраняет меньшую размерность, так как каждая категория заменяется одним числом (частотой). Хорошо работает с деревьями, которые нечувствительны к масштабу признаков.
- **Минусы:** Потеря информации о категориях, менее интерпретируемо.
- **Когда использовать:** Для больших датасетов или если много новых категорий в valid/test.

In [8]:
preprocessor_alt = ColumnTransformer(
    transformers=[
        ('cat', CountEncoder(handle_unknown=1), cat_cols)
    ], remainder='passthrough'
)

X_train_enc_alt = preprocessor_alt.fit_transform(X_train)
X_valid_enc_alt = preprocessor_alt.transform(X_valid)
X_test_enc_alt = preprocessor_alt.transform(X_test)

feature_names_alt = preprocessor_alt.get_feature_names_out()
X_train_enc_alt = pd.DataFrame(X_train_enc_alt, columns=feature_names_alt)
X_valid_enc_alt = pd.DataFrame(X_valid_enc_alt, columns=feature_names_alt)
X_test_enc_alt = pd.DataFrame(X_test_enc_alt, columns=feature_names_alt)

print(f'Альтернативный подход (CountEncoder) - Размер train: {X_train_enc_alt.shape}, valid: {X_valid_enc_alt.shape}, test: {X_test_enc_alt.shape}')

Альтернативный подход (CountEncoder) - Размер train: (24084, 32), valid: (24084, 32), test: (24815, 32)


### Релизация собственной функции для расчета Gini

In [9]:
def gini_score(y_true, y_pred_proba):
    if y_pred_proba.ndim == 1:
        auc = roc_auc_score(y_true, y_pred_proba)
    else:
        auc = roc_auc_score(y_true, y_pred_proba[:, 1])
    return 2 * auc - 1

## 2. Реализация собственного дерева решений (CART)
------

В этом блоке реализуем собственные классы DecisionTreeClassifier и DecisionTreeRegressor. Класс Node хранит данные и вычисляет нечистоту (impurity). Для классификации используем критерий Джини, для регрессии — стандартное отклонение. Также добавляем функцию для случайных разделений (для ExtraTrees).

**Класс Node**

In [10]:
class Node:
    def __init__(self, indices, depth=0):
        self.indices = indices
        self.depth = depth
        self.feature = None
        self.threshold = None
        self.left = None
        self.right = None
        self.value = None

    def gini(self, y):
        if len(self.indices) == 0:
            return 0
        p = np.bincount(y[self.indices].astype(int)) / len(self.indices)
        return 1 - np.sum(p**2)

    def std(self, y):
        return np.std(y[self.indices]) if len(self.indices) > 0 else 0

**Класс DecisionTreeBase**

In [11]:
class DecisionTreeBase:
    def __init__(self, max_depth=None, is_classifier=True, min_samples_split=2):
        self.max_depth = max_depth
        self.is_classifier = is_classifier
        self.min_samples_split = min_samples_split
        self.root = None
        self.X = None
        self.y = None

    def _impurity(self, node, y):
        return node.gini(y) if self.is_classifier else node.std(y)

    def _find_best_split(self, node):
        best_gain = -np.inf
        best_feature = None
        best_threshold = None

        for feature in range(self.X.shape[1]):
            values = np.unique(self.X[node.indices, feature])
            for threshold in values:
                left_idx = node.indices[self.X[node.indices, feature] <= threshold]
                right_idx = node.indices[self.X[node.indices, feature] > threshold]

                if len(left_idx) < self.min_samples_split or len(right_idx) < self.min_samples_split:
                    continue

                impurity_parent = self._impurity(node, self.y)
                impurity_left = (1 - np.sum((np.bincount(self.y[left_idx].astype(int))/len(left_idx))**2) if len(left_idx)>0 else 0) if self.is_classifier else (self.y[left_idx].std() if len(left_idx)>0 else 0)
                impurity_right = (1 - np.sum((np.bincount(self.y[right_idx].astype(int))/len(right_idx))**2) if len(right_idx)>0 else 0) if self.is_classifier else (self.y[right_idx].std() if len(right_idx)>0 else 0)

                gain = impurity_parent - (
                    len(left_idx)/len(node.indices) * impurity_left +
                    len(right_idx)/len(node.indices) * impurity_right
                )

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold, best_gain

    def _build_tree(self, node):
        #print(f"Глубина: {node.depth}, Размер узла: {len(node.indices)}")
        if (node.depth == self.max_depth or
            len(np.unique(self.y[node.indices])) == 1 or
            len(node.indices) < self.min_samples_split):
            if self.is_classifier:
                node.value = np.bincount(self.y[node.indices].astype(int), minlength=2) / len(node.indices)
            else:
                node.value = np.mean(self.y[node.indices])
            return

        feature, threshold, gain = self._find_best_split(node)
        if feature is None or gain <= 0:
            if self.is_classifier:
                node.value = np.bincount(self.y[node.indices].astype(int), minlength=2) / len(node.indices)
            else:
                node.value = np.mean(self.y[node.indices])
            return

        node.feature = feature
        node.threshold = threshold

        left_idx = node.indices[self.X[node.indices, feature] <= threshold]
        right_idx = node.indices[self.X[node.indices, feature] > threshold]

        node.left = Node(left_idx, node.depth + 1)
        node.right = Node(right_idx, node.depth + 1)

        self._build_tree(node.left)
        self._build_tree(node.right)

    def fit(self, X, y):
        self.X = np.array(X)
        self.y = np.array(y)
        self.root = Node(np.arange(len(self.y)))
        self._build_tree(self.root)
        self.X = None  # Очистка
        self.y = None
        gc.collect()

    def _predict_one(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self._predict_one(x, node.left)
        else:
            return self._predict_one(x, node.right)

    def predict_proba(self, X):
        if not self.is_classifier:
            raise ValueError("predict_proba только для классификатора")
        return np.array([self._predict_one(row, self.root) for row in np.array(X)])

    def predict(self, X):
        preds = np.array([self._predict_one(row, self.root) for row in np.array(X)])
        if self.is_classifier:
            return np.argmax(preds, axis=1) if len(preds.shape) > 1 else preds
        else:
            return preds

**Класс DecisionTreeClassifier**

In [12]:
class DecisionTreeClassifier(DecisionTreeBase):
    def __init__(self, max_depth=None, min_samples_split=2):
        super().__init__(max_depth, is_classifier=True, min_samples_split=min_samples_split)

**Класс DecisionTreeRegressor**

In [13]:
class DecisionTreeRegressor(DecisionTreeBase):
    def __init__(self, max_depth=None, min_samples_split=2):
        super().__init__(max_depth, is_classifier=False, min_samples_split=min_samples_split)

**Функция для случайного разделения (для ExtraTrees)**

In [27]:
def find_random_split(node, X, y, is_classifier):
    best_gain = -np.inf
    best_feature = None
    best_threshold = None

    for feature in range(X.shape[1]):
        values = np.unique(X[node.indices, feature])
        if len(values) < 2:
            continue
        threshold = np.random.choice(values)
        left_idx = node.indices[X[node.indices, feature] <= threshold]
        right_idx = node.indices[X[node.indices, feature] > threshold]

        if len(left_idx) < 2 or len(right_idx) < 2:
            continue

        impurity_parent = node.gini(y) if is_classifier else node.std(y)
        impurity_left = (1 - np.sum((np.bincount(y[left_idx].astype(int))/len(left_idx))**2) if len(left_idx)>0 else 0) if is_classifier else (y[left_idx].std() if len(left_idx)>0 else 0)
        impurity_right = (1 - np.sum((np.bincount(y[right_idx].astype(int))/len(right_idx))**2) if len(right_idx)>0 else 0) if is_classifier else (y[right_idx].std() if len(right_idx)>0 else 0)

        gain = impurity_parent - (len(left_idx)/len(node.indices) * impurity_left + len(right_idx)/len(node.indices) * impurity_right)

        if gain > best_gain:
            best_gain = gain
            best_feature = feature
            best_threshold = threshold

    return best_feature, best_threshold, best_gain

## 3. Оценка собственного дерева на валидационной выборке
------

Обучаем DecisionTreeClassifier на тренировочных данных и вычисляем коэффициент Джини на валидационной выборке. При необходимости настраиваем max_depth для достижения Джини >= 0.1.

In [ ]:
# Проверка данных
print("Классы в y_train:", np.unique(y_train))
print("Классы в y_valid:", np.unique(y_valid))
print("Пример X_train_enc:", X_train_enc.head())
print("Доля ненулевых значений в X_train_enc:", np.mean(X_train_enc.values != 0))

# Обучение
model = DecisionTreeClassifier(max_depth=7, min_samples_split=2)
model.fit(X_train_enc.values, y_train.values)

# Предсказание
proba_valid = model.predict_proba(X_valid_enc.values)
print("Пример вероятностей:", proba_valid[:5])  # Диагностика
gini_custom = gini_score(y_valid, proba_valid)
print(f'Джини собственного DT на валидации: {gini_custom}')

Классы в y_train: [0 1]
Классы в y_valid: [0 1]
Пример X_train_enc:    cat__Auction_ADESA  cat__Auction_MANHEIM  cat__Auction_OTHER  \
0                 0.0                   1.0                 0.0   
1                 0.0                   1.0                 0.0   
2                 0.0                   1.0                 0.0   
3                 0.0                   1.0                 0.0   
4                 0.0                   1.0                 0.0   

   cat__Make_ACURA  cat__Make_BUICK  cat__Make_CADILLAC  cat__Make_CHEVROLET  \
0              0.0              0.0                 0.0                  0.0   
1              0.0              0.0                 0.0                  0.0   
2              0.0              0.0                 0.0                  0.0   
3              0.0              0.0                 0.0                  1.0   
4              0.0              0.0                 0.0                  0.0   

   cat__Make_CHRYSLER  cat__Make_DODGE  cat__Mak

## 4. Сравнение с DecisionTreeClassifier из sklearn
------

Используем DecisionTreeClassifier из sklearn для сравнения. Проверяем, лучше ли он собственной реализации.

In [ ]:
from sklearn.tree import DecisionTreeClassifier as SkDT

sk_model = SkDT(max_depth=7, criterion='gini', random_state=42)
sk_model.fit(X_train_enc, y_train)

sk_proba = sk_model.predict_proba(X_valid_enc)
sk_gini = gini_score(y_valid, sk_proba)
print(f'Джини sklearn DT на валидации: {sk_gini}')

Джини sklearn DT на валидации: 0.43332538187431724


Резюме результатов: Sklearn обычно лучше благодаря оптимизированному поиску разделений и реализации на C++. Собственная версия медленнее и менее эффективна.

## 5. Реализация собственного RandomForestClassifier
------

Реализуем Random Forest с использованием бэггинга и случайного выбора признаков. Устанавливаем фиксированный random seed.

In [15]:
class RandomForestClassifier:
    def __init__(self, n_trees=20, max_depth=5, max_features=10, random_seed=42):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_seed = random_seed
        self.trees = []

    def fit(self, X, y):
        np.random.seed(self.random_seed)
        n_samples, n_feats = X.shape
        mf = min(self.max_features, n_feats)  # Ограничение признаков

        for i in range(self.n_trees):
            print(f"Обучение дерева {i+1}/{self.n_trees}")
            idx = np.random.choice(n_samples, n_samples, replace=True)
            X_boot = X[idx]
            y_boot = y[idx]

            feat_idx = np.random.choice(n_feats, mf, replace=False)
            X_boot_sub = X_boot[:, feat_idx]

            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_split=2)
            tree.fit(X_boot_sub, y_boot)
            self.trees.append((tree, feat_idx))
            gc.collect()  # Очистка памяти

    def predict_proba(self, X):
        probs = []
        for tree, feat_idx in self.trees:
            probs.append(tree.predict_proba(X[:, feat_idx]))
        return np.mean(probs, axis=0)

Обучение и оценка

In [ ]:
# Проверка использования памяти
print(f"RAM usage before RF: {psutil.virtual_memory().percent}%")

# Обучение и оценка
rf = RandomForestClassifier(n_trees=30, max_depth=7, max_features=10, random_seed=42)
rf.fit(X_train_enc.values, y_train.values)
rf_proba = rf.predict_proba(X_valid_enc.values)
rf_gini = gini_score(y_valid, rf_proba)
print(f"RAM usage after RF: {psutil.virtual_memory().percent}%")
print(f'Джини собственного RF на валидации: {rf_gini}')

RAM usage before RF: 21.7%
Обучение дерева 1/30
Обучение дерева 2/30
Обучение дерева 3/30
Обучение дерева 4/30
Обучение дерева 5/30
Обучение дерева 6/30
Обучение дерева 7/30
Обучение дерева 8/30
Обучение дерева 9/30
Обучение дерева 10/30
Обучение дерева 11/30
Обучение дерева 12/30
Обучение дерева 13/30
Обучение дерева 14/30
Обучение дерева 15/30
Обучение дерева 16/30
Обучение дерева 17/30
Обучение дерева 18/30
Обучение дерева 19/30
Обучение дерева 20/30
Обучение дерева 21/30
Обучение дерева 22/30
Обучение дерева 23/30
Обучение дерева 24/30
Обучение дерева 25/30
Обучение дерева 26/30
Обучение дерева 27/30
Обучение дерева 28/30
Обучение дерева 29/30
Обучение дерева 30/30
RAM usage after RF: 22.1%
Джини собственного RF на валидации: 0.37846974201500316


Резюме результатов: Улучшает результат одиночного дерева:

- На одиночном - Gini: 0.20203713111439425

- На RandomForest - Gini: 0.37846974201500316

## 6. Реализация собственного GBDT Classifier
------

Реализуем градиентный бустинг, используя регрессионные деревья для градиентов (BCE loss). Используем learning rate и выборку признаков.

In [16]:
class GBDTClassifier:
    def __init__(self, n_trees=100, max_depth=3, max_features='sqrt', lr=0.1, random_seed=42):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.max_features = max_features
        self.lr = lr
        self.random_seed = random_seed
        self.trees = []
        self.init_pred = None

    def fit(self, X, y):
        np.random.seed(self.random_seed)
        y = y.astype(float)
        self.init_pred = np.log(np.mean(y) / (1 - np.mean(y) + 1e-10))
        F = np.full(len(y), self.init_pred)

        for _ in range(self.n_trees):
            p = scipy.special.expit(F)
            residuals = y - p

            n_feats = X.shape[1]
            mf = int(np.sqrt(n_feats)) if self.max_features == 'sqrt' else self.max_features
            feat_idx = np.random.choice(n_feats, mf, replace=False)
            X_sub = X[:, feat_idx]

            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X_sub, residuals)
            self.trees.append((tree, feat_idx))

            tree_pred = tree.predict(X_sub)
            F += self.lr * tree_pred

    def predict_proba(self, X):
        F = np.full(X.shape[0], self.init_pred)
        for tree, feat_idx in self.trees:
            F += self.lr * tree.predict(X[:, feat_idx])
        p = scipy.special.expit(F)
        return np.vstack((1 - p, p)).T

Обучение и оценка

In [ ]:
# Обучение и оценка
gbdt = GBDTClassifier(n_trees=100, max_depth=3, lr=0.1)
gbdt.fit(X_train_enc.values, y_train.values)
gbdt_proba = gbdt.predict_proba(X_valid_enc.values)
gbdt_gini = gini_score(y_valid, gbdt_proba)
print(f'Джини собственного GBDT на валидации: {gbdt_gini}')

Джини собственного GBDT на валидации: 0.4502569959573457


## 7. Современные реализации GBDT (LightGBM, CatBoost, XGBoost)
------

Обучаем и сравниваем LightGBM, CatBoost и XGBoost с настройкой параметров. Отмечаем различия в обработке данных (например, CatBoost для категориальных признаков).

In [20]:
# Функция для обучения и оценки
def train_and_eval_lib(model_type, X_train, y_train, X_valid, y_valid):
    if model_type == 'lgb':
        dtrain = lgb.Dataset(X_train, y_train)
        dvalid = lgb.Dataset(X_valid, y_valid)
        params = {'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.1, 'max_depth': 6, 'seed': 42}
        bst = lgb.train(params, dtrain, num_boost_round=1000, valid_sets=[dvalid], callbacks=[lgb.early_stopping(50)])
        proba = bst.predict(X_valid)
    elif model_type == 'cb':
        model = cb.CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.1, eval_metric='AUC', random_seed=42)
        model.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=50, verbose=0)
        proba = model.predict_proba(X_valid)[:, 1]
    elif model_type == 'xgb':
        dtrain = xgb.DMatrix(X_train, y_train)
        dvalid = xgb.DMatrix(X_valid, y_valid)
        params = {'objective': 'binary:logistic', 'eval_metric': 'auc', 'learning_rate': 0.1, 'max_depth': 6, 'seed': 42}
        bst = xgb.train(params, dtrain, num_boost_round=1000, evals=[(dvalid, 'valid')], early_stopping_rounds=50, verbose_eval=False)
        proba = bst.predict(dvalid)

    gini = gini_score(y_valid, proba)
    return gini

Запуск и оценка

In [21]:
lgb_gini = train_and_eval_lib('lgb', X_train_enc, y_train, X_valid_enc, y_valid)
cb_gini = train_and_eval_lib('cb', X_train_enc, y_train, X_valid_enc, y_valid)
xgb_gini = train_and_eval_lib('xgb', X_train_enc, y_train, X_valid_enc, y_valid)

print(f'LightGBM Джини: {lgb_gini}')
print(f'CatBoost Джини: {cb_gini}')
print(f'XGBoost Джини: {xgb_gini}')

[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=6) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=64) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=6) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=64) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Info] Number of positive: 2756, number of negative: 21328
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003209 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Tota

Лучшая модель по итогу - XGBoost с показателем **Gini:**

**0.49174692122069263**

## 8. Оценка лучшей модели на тестовой выборке
------

Выбираем XGBoost, обучаем заново на тренировочной выборке и вычисляем Джини на train/valid/test. Проверяем на переобучение.

In [23]:
print("Классы в y_train:", np.unique(y_train, return_counts=True))
print("Классы в y_valid:", np.unique(y_valid, return_counts=True))
print("Классы в y_test:", np.unique(y_test, return_counts=True))
print(f"Размер X_train_enc: {X_train_enc.shape}")
print(f"RAM usage before training: {psutil.virtual_memory().percent}%")

# Приведение типов
y_train = y_train.astype(int)
y_valid = y_valid.astype(int)
y_test = y_test.astype(int)
X_train_enc = X_train_enc.astype(float)
X_valid_enc = X_valid_enc.astype(float)
X_test_enc = X_test_enc.astype(float)

# Обучение XGBoost
dtrain = xgb.DMatrix(X_train_enc, y_train)
dvalid = xgb.DMatrix(X_valid_enc, y_valid)
dtest = xgb.DMatrix(X_test_enc, y_test)
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.1,
    'max_depth': 6,
    'seed': 42
}
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=[(dvalid, 'valid')],
    early_stopping_rounds=50,
    verbose_eval=False
)

# Предсказания
train_proba = best_model.predict(dtrain)
valid_proba = best_model.predict(dvalid)
test_proba = best_model.predict(dtest)

# Проверка вероятностей
print("Пример вероятностей на train (первые 5):", train_proba[:5])
print("Пример вероятностей на valid (первые 5):", valid_proba[:5])
print("Пример вероятностей на test (первые 5):", test_proba[:5])
print("Уникальные вероятности на valid:", np.unique(valid_proba))

# Вычисление Джини
train_gini = gini_score(y_train, train_proba)
valid_gini = gini_score(y_valid, valid_proba)
test_gini = gini_score(y_test, test_proba)

print(f"XGBoost (лучшая модель) — Джини на train: {train_gini}")
print(f"Джини на valid: {valid_gini}")
print(f"Джини на test: {test_gini}")
print(f"RAM usage after training: {psutil.virtual_memory().percent}%")

# Очистка памяти
gc.collect()

Классы в y_train: (array([0, 1]), array([21328,  2756]))
Классы в y_valid: (array([0, 1]), array([20926,  3158]))
Классы в y_test: (array([0, 1]), array([21753,  3062]))
Размер X_train_enc: (24084, 1713)
RAM usage before training: 27.7%
Пример вероятностей на train (первые 5): [0.04979338 0.06662174 0.12935837 0.05134354 0.10736223]
Пример вероятностей на valid (первые 5): [0.02458949 0.04551674 0.0472426  0.09970537 0.03983329]
Пример вероятностей на test (первые 5): [0.10006604 0.08838389 0.09167034 0.03559981 0.04693903]
Уникальные вероятности на valid: [0.0065335  0.00676447 0.00701582 ... 0.9775835  0.9779408  0.98158205]
XGBoost (лучшая модель) — Джини на train: 0.7128217898995794
Джини на valid: 0.49174692122069263
Джини на test: 0.5065977821238228
RAM usage after training: 37.5%


85

## 9. Реализация ExtraTreesClassifier
-------------------------

Реализуем ExtraTrees: без бутстрапа по строкам, с случайными разделениями.

In [28]:
class ExtraTreesClassifier:
    def __init__(self, n_trees=20, max_depth=5, max_features=10, random_seed=42):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_seed = random_seed
        self.trees = []

    def fit(self, X, y):
        np.random.seed(self.random_seed)
        n_samples, n_feats = X.shape
        mf = min(self.max_features, n_feats)  # Ограничение признаков

        for i in range(self.n_trees):
            #print(f"Обучение дерева {i+1}/{self.n_trees}")
            feat_idx = np.random.choice(n_feats, mf, replace=False)
            X_sub = X[:, feat_idx]

            # Определяем ExtraTreeClassifier с переопределённым _find_best_split
            class ExtraTreeClassifier(DecisionTreeClassifier):
                def _find_best_split(self, node):
                    return find_random_split(node, self.X, self.y, is_classifier=True)

            tree = ExtraTreeClassifier(max_depth=self.max_depth, min_samples_split=2)
            tree.fit(X_sub, y)
            self.trees.append((tree, feat_idx))
            gc.collect()  # Очистка памяти

    def predict_proba(self, X):
        probs = []
        for tree, feat_idx in self.trees:
            probs.append(tree.predict_proba(X[:, feat_idx]))
        return np.mean(probs, axis=0)

Обучение и оценка.

In [29]:
print(f"RAM usage before ExtraTrees: {psutil.virtual_memory().percent}%")

# Обучение и оценка
et = ExtraTreesClassifier(n_trees=20, max_depth=5, max_features=10, random_seed=42)
et.fit(X_train_enc.values, y_train.values)
et_proba = et.predict_proba(X_valid_enc.values)
et_gini = gini_score(y_valid, et_proba)

print("Пример вероятностей ExtraTrees (первые 5):", et_proba[:5])
print("Уникальные вероятности класса 1:", np.unique(et_proba[:, 1]))
print(f"Джини ExtraTrees на валидации: {et_gini}")
print(f"RAM usage after ExtraTrees: {psutil.virtual_memory().percent}%")

# Очистка памяти
gc.collect()

RAM usage before ExtraTrees: 37.2%
Пример вероятностей ExtraTrees (первые 5): [[0.8911778  0.1088222 ]
 [0.89637086 0.10362914]
 [0.88825904 0.11174096]
 [0.88671634 0.11328366]
 [0.88285412 0.11714588]]
Уникальные вероятности класса 1: [0.09755338 0.09833466 0.09847226 ... 0.15376495 0.15609039 0.1566234 ]
Джини ExtraTrees на валидации: 0.2467177835924379
RAM usage after ExtraTrees: 37.2%


0